In [1]:
import numpy as np
import pandas as pd

import wrangle as w

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, \
    confusion_matrix,\
    classification_report

In [2]:
readmes = w.wrangle_github_repositories()

In [3]:
readmes.head(3)

,repo,language,readme_contents,cleaned_readme_contents
0,huggingface/transformers,Python,<!---\nCopyright 2020 The HuggingFace Team. Al...,copyright 2020 huggingface team right reserved...
1,apachecn/ailearning,Python,"<p align=""center"">\n <a href=""https://www.a...",p aligncenter hrefhttpswwwapachecnorg img widt...
2,google-research/bert,Python,"# BERT\n\n**\*\*\*\*\* New March 11th, 2020: S...",bert new march 11th 2020 smaller bert model re...


In [7]:
# remove null rows (adding this to prep stage)
readmes = readmes[readmes.cleaned_readme_contents.isna() == False]

## Split data

In [8]:
train, validate, test = w.train_split(readmes)

In [9]:
readmes.shape, train.shape, validate.shape, test.shape

((493, 4), (275, 4), (119, 4), (99, 4))

In [13]:
X_train = train.cleaned_readme_contents
y_train = train.language
X_validate = validate.cleaned_readme_contents
y_validate = validate.language
X_test = test.cleaned_readme_contents
y_test = test.language

## Baseline

In [19]:
train.language.value_counts(normalize=True)

Python    0.610909
HTML      0.389091
Name: language, dtype: float64

In [20]:
baseline = 'Python'

## The baseline is 'Python' with an accuacy of 61%

# Modeling

In [14]:
# make a Count Vectorizer object
cv = CountVectorizer()
# use the thing
X_bow = cv.fit_transform(X_train)
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train)

0.8836363636363637

In [11]:
bag_of_words_cv

<275x36883 sparse matrix of type '<class 'numpy.int64'>'
	with 85429 stored elements in Compressed Sparse Row format>

In [12]:
# make a Tfidf Vectorizer object
tfidf = TfidfVectorizer()
# use the thing
bag_of_words_tfidf = tfidf.fit_transform(X_train)

In [ ]:
cv = CountVectorizer(ngram_range=(2, 3))
bag_of_grams = cv.fit_transform(X_train)

In [ ]:
tfidf = CountVectorizer(ngram_range=(2, 3))
bag_of_grams = tfidf.fit_transform(X_train)

In [ ]:
def model_metrics(model,
                  model_name,
                  X_train, 
                  y_train, 
                  X_val, 
                  y_val, 
                  scores=metric_df):
    '''
    model_metrics will use an sklearn model object to 
    create predictions after fitting on our training set, and add
    the model scores to a pre-established metric_df
    returns: metric_df
    **TODO: create a check to see if metric_df exists.  
    Create it if not
    '''
    # fit our model object
    model.fit(X_train, y_train['Infant.Mortality'])
    in_sample_pred = model.predict(X_train)
    out_sample_pred = model.predict(X_val)
#     model_name = input('Name for model? python identifer friendly pls.\n')
    y_train[model_name] = in_sample_pred
    y_val[model_name] = out_sample_pred
    print(y_val.shape)
    print(out_sample_pred.shape)
    rmse_val = mean_squared_error(
    y_val['Infant.Mortality'], out_sample_pred, squared=False)
    r_squared_val = explained_variance_score(
        y_val['Infant.Mortality'], out_sample_pred)
    return metric_df.append({
        'model': model_name,
        'rmse': rmse_val,
        'r^2': r_squared_val
    
    }, ignore_index=True)